### Essaies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_blobs

def SVM(Gamma, tolerance, nbr_points, nbr_centres, proba_ajoutee):

    X, y = make_blobs(n_samples=nbr_points, centers=nbr_centres, random_state=proba_ajoutee)
    clf = svm.SVC(kernel='rbf', gamma = Gamma, C=tolerance)
    clf.fit(X, y)
    plt.scatter(X[:, 0], X[:, 1], c='black')
    plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], c='red')
SVM(0.0001, 1000000000, 50, 2, 4)

# Codes ébauche SVM

Version "Ludique"

In [ ]:
#L'objectif est de prédire l'issue d'une nouvelle entrée à l'aide de données d'entrainement et d'un classificateur SVM.
##########################################DEBUT DU CODE##########################################
#On importe les librairies. 
import numpy as np
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder

#On créer un fonction qui permet de vectoriser une nouvelle entrée à partir de la base de données de référence. 
def vectorise(BDD, prediction):
    
    nbr_categorie=len(prediction)
    entree=[[i] for i in prediction]
    P=[[] for j in range(0, nbr_categorie)]
    Pfinal=[[] for j in range(0, nbr_categorie)]
    Vector=[[] for j in range(0, nbr_categorie)]
    
    for i in BDD:
        for j in range(0, nbr_categorie):
            P[j].append(i[j])
#Les entrées sont à présent regroupées dans des sous-listes selon la catégorie à laquelle elles appartiennent. 
            
            for u in P[j]:
                if u not in Pfinal[j]:
                    Pfinal[j].append(u)   
#Enlève toutes les récurrences dans les listes. 
            Pfinal[j]=sorted(Pfinal[j])
#Trie par ordre croissant les entrées
            
    for w in range(0, nbr_categorie):
        for i in entree[w]:
            for j in Pfinal[w]:
                if i==j:
                    Vector[w].append(1)
                else:
                    Vector[w].append(0)
#Vectorise l'entrée. 
        
    return np.array([j for i in Vector for j in i])

#L'issue de cette fonction est une matrice vectorisée de la même manière que OneHotEncoder. 
#On créer la fonction du classificateur. 

def ClassificateurLudique(BDD,Y, prediction):
    
#On initialise le vectorisateur.  
    enc = OneHotEncoder(handle_unknown='ignore')
#On vectorise la base de données via OneHotEncoder.
    X = enc.fit_transform(BDD).toarray()
#On initialise le SVM
    clf = svm.SVC(kernel='rbf', gamma=0.001, C=1000)
#Entrainement du SVM grace aux points d'entrainement vectorisés au préalable. 
    clf.fit(X, Y)
#ATTENTION, la base de données à fournir à la fonction vectorise, ne doit pas être vectorisée ! 
    return(clf.predict([vectorise(BDD, prediction)]))
#Retourne un la prédiction de la nouvelle entrée, elle aussi vectorisée au préalable. 

##########################################FIN DU CODE##########################################

In [ ]:
A=[['femme', 'blond', 'petit', 'lunettes'], 
   ['homme', 'chatin', 'petit', 'lunettes'], 
   ['homme', 'blond', 'grand', 'lunettes'], 
   ['femme', 'chatin','grand', 'pas lunettes'], 
   ['homme', 'brun','grand', 'pas lunettes']]

B=['Marie', 'Jonas', 'Julien', '(Marie)^-1', 'Damien']

P=['homme', 'chatin','grand', 'pas lunettes']
   
ClassificateurLudique(A, B, P)

Version Numérique

In [ ]:
#L'objectif est de prédire l'issue d'une nouvelle entrée à l'aide de données d'entrainement et d'un classificateur SVM.
##########################################DEBUT DU CODE##########################################
#On importe les librairies. 

import numpy as np
import sqlite3
from sklearn import svm

def ClassificateurNumerique(prediction):

#On se connecte à la base de données. 
    bdd=sqlite3.connect('C:\Users\Stanislas Brusselle\Desktop\TIPE\BDD\BDD_virus.db')
    
#Création de Liste1 qui régit à des données sur certains virus.    
    requete1 = """SELECT R0, Virulence, Incubation FROM Classification_virus"""
    X = [ e for e in bdd.execute(requete1).fetchall()]
    
#Création de Liste2 qui définit le virus lié aux données de Liste1.   
    requete2 = """SELECT Virus FROM Classification_virus"""
    Y = [i for i in bdd.execute(requete2).fetchall()]

#Fermeture de la base de données. 
    bdd.close

#Initialisation d'un SVM
    clf = svm.SVC(kernel='rbf', gamma=0.001, C=1000)
#Entrainement du SVM grace aux listes 1 et 2.    
    clf.fit(X, Y)
    
    return(clf.predict([prediction]))

def Prediction():
    
    R0 = input("R0 : ")
    Virulence = input("Virulence :  ")
    Incubation = input("temps d'incubation : ")
    
    return (ClassificateurNumerique([R0,Virulence, Incubation])) 

In [ ]:
Prediction()

# Déterminer un nouveau virus 

In [ ]:
#L'objectif est de prédire l'issue d'une nouvelle entrée à l'aide de données d'entrainement et d'un classificateur SVM.
##########################################DEBUT DU CODE##########################################
#On importe les librairies. 

from sklearn.metrics import accuracy_score
import numpy as np
import sqlite3
from sklearn import svm


def BDD():
#On se connecte à la base de données. 
    bdd=sqlite3.connect('C:\Users\Stanislas Brusselle\Desktop\TIPE\BDD\BDD_virus.db')
    
#Création de Liste1 qui régit à des données sur certains virus.    
    requete1 = """SELECT R0, Virulence, Incubation FROM Classification_virus"""
    X = [ e for e in bdd.execute(requete1).fetchall()]
    
#Création de Liste2 qui définit le virus lié aux données de Liste1.   
    requete2 = """SELECT Virus FROM Classification_virus"""
    Y = [i for i in bdd.execute(requete2).fetchall()]

#Fermeture de la base de données. 
    bdd.close
    return([X, Y])



def ClassificateurNumerique(prediction):

    BDD()
    X=BDD()[0]
    Y=BDD()[1]
    
#Initialisation d'un SVM à kernel RBF
    clf = svm.SVC(kernel='rbf', gamma=0.001, C=1000, probability=True)
#Entrainement du SVM grace aux listes 1 et 2.    
    clf.fit(X, Y)
    
    predic=clf.predict([prediction])

#Ici commence la partie pour prédire si le virus est connu de la base de données ou non

    B=[accuracy_score(i,predic, normalize=True) for i in Y]
# Ajoute 1 si c'est la valeur prédite par l'algorithme, 0 dans le cas contraire
    W=clf.predict_proba([prediction]) 
#Plus la probabilité est élevée, moins il y a de chances que l'algorithme ait raison. 

    R=[]
    for i in range(0, len(B)):
        if B[i] == 1.0:
            R.append(i)
                
#incertitude seuil a 10%           
    if W[0][R[0]]>=0.1:
        return('Virus inconnu a la base de donnees')
    else:
        return(predic, W[0][R[0]])

    
def Prediction():
    
#caractéristique de la pandémie
    R0 = input("R0 : ")
    Virulence = input("Virulence :  ")
    Incubation = input("temps d'incubation : ")
    
#retourne sur la nature de la pandémie. 
    return(ClassificateurNumerique([R0,Virulence, Incubation]))

##########################################FIN DU CODE##########################################

In [ ]:
Prediction()

# Modifier la base de données si virus inconnu 


In [ ]:
import numpy as np
import sqlite3
from sklearn import svm


def ClassificateurNumerique(prediction):
#On se connecte à la base de données. 
    bdd=sqlite3.connect('C:\Users\Stanislas Brusselle\Desktop\TIPE\BDD\BDD_virus.db')
    
#Création de Liste1 qui régit à des données sur certains virus.    
    requete1 = """SELECT R0, Virulence, Incubation FROM Classification_virus"""
    X = [ e for e in bdd.execute(requete1).fetchall()]
    
#Création de Liste2 qui définit le virus lié aux données de Liste1.   
    requete2 = """SELECT Virus FROM Classification_virus"""
    Y = [i for i in bdd.execute(requete2).fetchall()]
    
#Initialisation d'un SVM à kernel RBF
    clf = svm.SVC(kernel='rbf', gamma=0.001, C=1000, probability=True)
#Entrainement du SVM grâce aux listes 1 et 2.    
    clf.fit(X, Y)
    
    predic=clf.predict([prediction])

#Ici commence la partie pour prédire si le virus est connu de la base de données ou non
    
    W=clf.predict_proba([prediction]) 
    print(W)
    
    a=W[0][0]
    rang=0   
    for i in range(0, len(W[0])):
         if a>=W[0][i]:
                a=W[0][i]
                rang=i  
    
    if W[0][rang]>=0.09:
        print('Virus inconnu a la base de donnees')
        c=bdd.cursor()
        c.execute('INSERT INTO Classification_virus VALUES (?,?,?,?)', (raw_input("Nom: "), prediction[0], prediction[1], prediction[2]))
        bdd.commit()
        bdd.close()
        print("Le virus a bien ete enregistre")
    else:
        bdd.close()
        return(predic, W[0][rang])

def Prediction():
    
#caractéristique de la pandémie
    R0 = input("R0 : ")
    Virulence = input("Virulence :  ")
    Incubation = input("temps d'incubation : ")
    
#retourne la nature de la pandémie. 
    return(ClassificateurNumerique([R0,Virulence, Incubation]))

##########################################FIN DU CODE##########################################

In [ ]:
Prediction()

# Optimisation de l'algorithme par le choix des paramètres 

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import sqlite3
import numpy as np

def KernelOpt(X, Y):
    cv = StratifiedKFold(n_splits=len(Y)//5)
    parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10], 'gamma':[0, 10]}
    svc = svm.SVC()
    Opt=GridSearchCV(svc, parameters, cv=cv)
    Opt.fit(X, Y)
    return([np.array(Opt.best_params_), Opt.best_score_])

def BDD():

    bdd=sqlite3.connect('C:\Users\Stanislas Brusselle\Desktop\TIPE\BDD\BDD_virus.db')
        
    requete1 = """SELECT R0, Virulence, Incubation FROM Classification_virus"""
    X = np.array([ e for e in bdd.execute(requete1).fetchall()])
     
    requete2 = """SELECT Virus FROM Classification_virus"""
    Y = [i for i in bdd.execute(requete2).fetchall()]

    bdd.close
    return([X, Y])

KernelOpt(BDD()[0], BDD()[1])

In [ ]:
import numpy as np
import sqlite3
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV, KFold
import time

def BDD():

    bdd=sqlite3.connect('C:\Users\Stanislas Brusselle\Desktop\TIPE\BDD\BDD_virus.db')
        
    requete1 = """SELECT R0, Virulence, Incubation FROM Classification_virus"""
    X = np.array([ e for e in bdd.execute(requete1).fetchall()])
     
    requete2 = """SELECT Virus FROM Classification_virus"""
    Y = [i for i in bdd.execute(requete2).fetchall()]

    bdd.close
    return([X, Y])



def ClassificateurNumerique(prediction):

    clf = svm.SVC(kernel='rbf', gamma= ,C= , probability=True)    
    clf.fit(BBD()[0], BDD()[1])
    predic=clf.predict([prediction])
    
#Ici commence la partie pour prédire si le virus est connu de la base de données ou non

#Renvoie les proba de d'incertitude liées à chaques catégories
    W=clf.predict_proba([prediction]) 
    print(W)
#Séletionne la proba la plus élévée et sa place dans la liste. 
    a=W[0][0]
    rang=0   
    for i in range(0, len(W[0])):
         if a>=W[0][i]:
                a=W[0][i]
                rang=i  
#Si la proba est plus élévée que 0.09 alors la prédiction est considérée comme nouveau virus.   
    if W[0][rang]>=0.09:
        print('Virus inconnu a la base de donnees')
#Ajoute le virus à la base de données. 
        c=bdd.cursor()
        c.execute('INSERT INTO Classification_virus VALUES (?,?,?,?)', (raw_input("Nom: "), prediction[0], prediction[1], prediction[2]))
        bdd.commit()
        bdd.close()
        print("Le virus a bien ete enregistre")
    else:
        bdd.close()
        return(predic, W[0][rang])

def Prediction():
    
#caractéristique de la pandémie à déterminer. 
    R0 = input("R0 : ")
    Virulence = input("Virulence :  ")
    Incubation = input("temps d'incubation : ")
    
#retourne la nature de la pandémie. 
    return(ClassificateurNumerique([R0,Virulence, Incubation]))

##########################################FIN DU CODE##########################################

In [ ]:
Prediction()

# Détermination complexité 

In [ ]:
import numpy as np
import sqlite3
from sklearn import svm
import time
import matplotlib.pyplot as plt


def BDD(taille):

    bdd=sqlite3.connect('C:\Users\Stanislas Brusselle\Desktop\TD BDD IPT\Fichiers Bases de données\geographie.sqlite')  
    requete1 = """SELECT surface, longitude, latitude, population_2010 FROM communes"""
    X = [ e for e in bdd.execute(requete1).fetchmany(size=taille)]
    
    requete2 = """SELECT num_departement FROM communes"""
    Y = [i for i in bdd.execute(requete2).fetchmany(size=taille)] 
    bdd.close()
    
    return([X, Y])



def ClassificateurNumeriqueCOMP(prediction, taille):

    BDD(taille)
    X=BDD(taille)[0]
    Y=BDD(taille)[1]
    
    clf = svm.SVC(kernel='rbf', gamma=0.1, C=10, probability=True)  
    clf.fit(X, Y)
    predic=clf.predict([prediction])
    

    W=clf.predict_proba([prediction]) 
    
    a=W[0][0]
    rang=0   
    for i in range(0, len(W[0])):
         if a>=W[0][i]:
                a=W[0][i]
                rang=i  
    
    if W[0][rang]>=0.09:
        print('None')
    else:
        return(predic, W[0][rang])

    
def Complexite(taille, pas):
    
    init=425
    Temps=[]
    Data=[]
    
    while init<=taille:
        
        t1=time.time() 
        c= ClassificateurNumeriqueCOMP([6.6,2866, 618, 51546], init)
        t2=time.time()
        
        print(c, t2-t1, init )
        
        Data.append(init)
        Temps.append(t2-t1)
        
        init+=pas 
    
    plt.plot(Data, Temps)
    plt.show

In [ ]:
Complexite(taille = 4000, pas = 100)

# Code fait main

In [64]:
from math import*
import numpy as np
import sqlite3

def Categories(X, Y):
#ATTENTION : Y ne doit contenir que deux catégories distinctes.   
    CD1=[]
    CD2=[]
    
    for i in range(0, len(Y)):
        if Y[i]!=Y[0]:
            CD1.append(X[i])
            c=Y[i]
        else:
            CD2.append(X[i])
#Renvoie les valeurs de X triées selon leurs catégories. 
    return [[CD1, CD2],[c, Y[0]]]

def distanceH(x, n):
    return((sqrt(produit_scalaire(X,n)**2))/(sqrt(produit_scalaire(n,n))))

def distanceP(x, y):
    return (sqrt(produit_scalaire(x, y)))

def produit_scalaire(X, Y):
    c=0
    for i in range(0, len(X)):
           c+=X[i]*Y[i]
    return c

def norme(x):
    return sqrt(produit_scalaire(x, x))

def equahyperplan(barycentre, M, n):
    
    coord=np.array(barycentre)-np.array(M)
    result=0
    for i in range(0, len(n)):
        result+=np.array(n[i])*(coord[i])
    return result
        
def verifopt(x, y, barycentre, n):
    l1=0
    l2=0
    if equahyperplan(barycentre, x[0], n)>0: 
        for i in x :
            if equahyperplan(barycentre, i, n)>0:
                l1=1
            else :
                return False
        for j in y:
            if equahyperplan(barycentre, j, n)<0:
                l2=1
            else :
                return False
        if l2==1 and l1==1:
            return True
                
    elif equahyperplan(barycentre, x[0], n)<0:
        for i in x:
            if equahyperplan(barycentre, i, n)<0:
                l1=1
            else :
                return False
        for j in y:
            if equahyperplan(barycentre, j, n)>0:
                l2=1   
            else :
                return False
        if l2==1 and l1==1:
            return True
    
def BDD(taille):

    bdd=sqlite3.connect('C:\Users\Stanislas Brusselle\Desktop\TIPE\BDD\BDD_virus2.db')  
    requete1 = """SELECT notes_M, notes_PC FROM Eleve"""
    X = [ e for e in bdd.execute(requete1).fetchmany(size=taille)]
    
    requete2 = """SELECT Etudiants FROM Eleve"""
    Y = [i for i in bdd.execute(requete2).fetchmany(size=taille)] 
    bdd.close()
    
    return([X, Y])
        
                

def Predict(X, Y, M, accuracy):    
    
#Objectifs :   
#Je vais calculer les distance a chaque points des deux catégories puis déterminer les valeurs minimales de distance. 
#Enfin je vais faire passer un hyperplan séparant en deux ces distances trouvées.
#Il faudra ensuite effectuer une rotation de l'hyperplan autour d'un barycentre pour que la séparattion soit optimale. 

#Hypothèse simplificatrice: 
#Je supposerai dans un permier temps les données linéairement séparables. 
#Détermination du vecteur normal et du barycentre de rotation. 
    W=Categories(X, Y)
#ATTENTION: je définis x et y comme étant x les valeurs qui correspondent à la première classe et y celles de la deuxième. 
    x=W[0][0]
    y=W[0][1]
    label=W[1]
    
    c=distanceP(x[0], y[0])
    
    for i in x:
        for j in y:
            if distanceP(i, j)<=c:
                c=distanceP(i, j) 
                L=[i, j]
#Calcul de la distance minimale entre A et B, ci-dessus. 
#Détermination du barycentre de rotation de l'hyperplan   

    barycentre= np.array(L[1])-(c/2)
    n=np.array(L[1])-barycentre
    npremier=n
    
#Détermination de l'équation de l'hyperplan

    result = equahyperplan(barycentre, M, n)
#optimisation de l'hyperplan   
#J'optimise n dans le cas où l'hyperplan n'est pas optimal. 
    compteur=0
    while verifopt(x, y, barycentre, n)==False:
        n =np.array(n).dot(np.array([[cos(pi/accuracy), -sin(pi/accuracy)], [sin(pi/accuracy), cos(pi/accuracy)]])) 
        compteur+=1
        if compteur>=100:
            print('nbr itérations dépassé')
            n=npremier
            break
            
    if equahyperplan(barycentre, x[0], n)>0:
        
        if result>0:
            return(label[0])
        else :
            return(label[1])
        
    if equahyperplan(barycentre, x[0], n)<0:
        
        if result<0:
            return(label[0])
        else :
            return(label[1])     
        

In [65]:
X=BDD(10)[0]
Y=BDD(10)[1]
Predict(X, Y, [1, 18], 5)

nbr itérations dépassé


(u'Mauvais',)

# Complexité 

In [ ]:
import matplotlib.pyplot as plt
from random import*
import time

#Je commence d'abord par créer un base de données linéaireemnt séparable.
def BDD(sample):
    X=[]
    Y=[]
    for i in range(1, sample/2):
        X.append([1, randint(25, 40)]) 
        Y.append('U1')
    for j in range(1, sample/2):
        X.append([1, randint(1, 10)])
        Y.append('U2')
    return(X, Y)

#Je poursuis en créant une fonction qui test l'algorithme sur des BDD de différentes tailles en calculant le temps de calcul.
def complexite(essais, pas):
    init=20
    Data=[]
    Temps=[]
    c=0
    
    for i in range(1, essais):
        t1=time.time() 
        Predict(BDD(init)[0], BDD(init)[1], [1, 35])
        t2=time.time() 
        c+=1
        print(t2-t1, init, Predict(BDD(init)[0], BDD(init)[1], [1, 35]), c)
        
        Data.append(init)
        Temps.append(t2-t1)
        
        init=init+pas
        
    plt.plot(Data, Temps)
    plt.show        

# Orthonormalisation par la méthode de Gram-Schimdt :

In [ ]:
from math import*
import numpy as np

def produit_scalaire(X, Y):
    c=0
    for i in range(0, len(X)):
           c+=X[i]*Y[i]
    return c

def norme(x):
    return sqrt(produit_scalaire(x, x))

def projeterOrthBON(x, E):
    c=0
    for i in E:
        c+=produit_scalaire(x,i)*i
    return c

#J'utilise le produit scalaire usuel de R^n. 

def Gram_Schmidt(X):
    
    X=np.array(X)
    
    Xor=[X[0]/norme(X[0])]
    E=[X[0]/norme(X[0])]
    
    for i in range(0, len(X)-1):
        ek= X[i+1]-projeterOrthBON(X[i+1], E)
        Xor.append(ek/norme(ek))
        E.append(Xor[i])
    return Xor   

In [ ]:
Gram_Schmidt([[1, 1, -1], [3, 0, 1]])

In [ ]:
def verifBON(X):
    for i in range(0, len(X)):
        print(norme(X[i]))
#Une approximation est faite par l'ordinateur. 
  

In [ ]:
verifBON([[ 0.57735027,  0.57735027, -0.57735027],[5/sqrt(42), (-4)/sqrt(42), 1/sqrt(42)]])